# Distributed data parallel MNIST training with TensorFlow2 and SMDataParallel

SMDataParallel is a new capability in Amazon SageMaker to train deep learning models faster and cheaper. SMDataParallel is a distributed data parallel training framework for TensorFlow2, PyTorch and MXNet.  

This notebook example shows how to use SMDataParallel with TensorFlow2 in SageMaker using MNIST dataset.

For more information:
1. [TensorFlow in SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html)
2. [SMDataParallelTensorFlow API Specification] < LINK TO BE ADDED >
3. [Getting started with SMDataParallel on SageMaker] < LINK TO BE ADDED >

**NOTE:** This example requires SageMaker Python SDK v2.X.

### Dataset
This example uses the MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of handwritten digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).

### SageMaker execution roles

The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with the appropriate full IAM role arn string(s).


In [ ]:
!pip install sagemaker --upgrade
import IPython
install_needed = True  # should only be True once
if install_needed:
    print("installing deps and restarting kernel")
    !python3 -m pip install -U sagemaker
    !python3 -m pip install -U smdebug
    IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import (ProfilerConfig, 
                                FrameworkProfile, 
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig)

## Model training with SMDataParallel

### Training script

The `train_tensorflow_smdataparallel_mnist.py` script provides the code you need for training a SageMaker model using SMDataParallel's `DistributedGradientTape`. The training script is very similar to a TensorFlow2 training script you might run outside of SageMaker, but modified to run with SMDataParallel. SMDataParallel's TensorFlow client provides an alternative to native `DistributedGradientTape`. For details about how to use SMDataParallel in your native TF2 script, see the Getting started with SMDataParallel tutorial.


In [ ]:
!pygmentize code/train_tensorflow_smdataparallel_mnist.py

### SageMaker TensorFlow Estimator function options


In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [ ]:
profiler_config=ProfilerConfig(
    system_monitor_interval_millis=100,
    framework_profile_params=FrameworkProfile(
        local_path="/opt/ml/output/profiler/",
        detailed_profiling_config=DetailedProfilingConfig(
            start_step=1, 
            num_steps=1
        ),
        dataloader_profiling_config=DataloaderProfilingConfig(
            start_step=1, 
            num_steps=1
        ),
        python_profiling_config=PythonProfilingConfig(
            start_step=1, 
            num_steps=1
        )
    )
)

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
                        base_job_name='tensorflow2-smdataparallel-mnist',
                        source_dir='code',
                        entry_point='train_tensorflow_smdataparallel_mnist.py',
                        role=role,
                        py_version='py37',
                        framework_version='2.4.1',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=2,
                        # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                             }
                                      }}
                        )


# estimator = TensorFlow(
#                         base_job_name=training_job_name,
#                         source_dir='code',
#                         entry_point='train_tensorflow_smdataparallel_mnist.py',
#                         role=role,
#                         py_version='py37',
#                         framework_version='2.3.1',
#                         # For training with multinode distributed training, set this count. Example: 2
# #                         instance_count=2,
#                         instance_count=1,
#                         # For training with p3dn instance use - ml.p3dn.24xlarge
#                         instance_type= 'ml.p3.16xlarge',
#                         sagemaker_session=sagemaker_session,
#                         profiler_config=profiler_config,  
#                         # Training using SMDataParallel Distributed Training Framework
#                         # missing code !!!
#                         )


In [ ]:
estimator.fit()

## Next steps

Now that you have a trained model, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests. The following cell will store the model_data variable to be used with the inference notebook.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
import boto3 
training_job_name=estimator.latest_training_job.job_name
session=boto3.session.Session()
region=session.region_name
tj=TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()
from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame



In [ ]:
pf=PandasFrame(tj.profiler_s3_output_path)
system_metrics_df=pf.get_all_system_metrics()
framework_metrics_df=pf.get_all_framework_metrics()

In [ ]:
import os
from IPython.display import IFrame, display, Markdown
from smdebug.profiler.python_profile_utils import StepPhase, PythonProfileModes
from smdebug.profiler.analysis.utils.python_profile_analysis_utils import Metrics
from smdebug.profiler.analysis.python_profile_analysis import PyinstrumentAnalysis, cProfileAnalysis
from smdebug.profiler.analysis.utils.pandas_data_analysis import PandasFrameAnalysis, StatsBy, Resource

In [ ]:
pf_analysis=PandasFrameAnalysis(system_metrics_df, framework_metrics_df)

In [ ]:
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram

system_metrics_reader=tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

metrics_histogram=MetricsHistogram(system_metrics_reader)
metrics_histogram.plot(starttime=0, 
                       endtime=system_metrics_reader.get_timestamp_of_latest_available_file(), 
                       select_dimensions=["CPU", "GPU"],
                       select_events=["total"])

In [ ]:
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap

system_metrics_reader.refresh_event_file_list()
view_heatmap=Heatmap(system_metrics_reader, plot_height=900)

In [ ]:
from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame

pf=PandasFrame(tj.profiler_s3_output_path)

In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data